In [1]:
from glob import glob
import json
import re
import yaml
import os
import shutil
import time
os.chdir('../')

from api.couch_db import TweetsDB
from api.task_helpers import GeoAnalyser, SentimentAnalyser, UsersAnalyser, TopicsAnalyser

with open("config.yaml", 'r') as ymlfile:
    cfg = yaml.load(ymlfile)
    ymlfile.close()

queues_cfg = cfg['QUEUES']
tweets_queue_path = queues_cfg['new_tweets']
processed_tweets_path = queues_cfg['processed_tweets']
sa_queue_path = queues_cfg['sentiment_tasks']
geo_queue_path = queues_cfg['geo_tasks']
users_tasks_path = queues_cfg['user_tweets_tasks']
topic_tasks_path = queues_cfg['topic_tasks']

In [2]:
#unprocessed_tweets = glob('{}/*.json'.format(tweets_queue_path))
#len(unprocessed_tweets)

# Process Downloaded Tweets Queue

In [3]:
def complete_task(tweet, path):
    filename = path.split('/')[-1]
    new_path = '{}/{}'.format(processed_tweets_path, filename)
    shutil.move(path, new_path)

In [4]:
analysers = [GeoAnalyser(geo_queue_path), SentimentAnalyser(sa_queue_path),
             UsersAnalyser(users_tasks_path), TopicsAnalyser(topic_tasks_path)]
#analysers = [SentimentAnalyser(sa_queue_path)]
couch_db = TweetsDB(cfg['COUCHDB'])

In [5]:
i = 1
while True:
    unprocessed_tweets = glob('{}/*.json'.format(tweets_queue_path))
    for path in unprocessed_tweets:
        try:
            with open(path, 'r') as fp:
                tweet_json = json.load(fp)
                tweet_id = tweet_json['id_str']
                for analyser in analysers:
                    analyser.append_task(tweet_id, tweet_json)
                couch_db.save_tweet(tweet_json)
                complete_task(tweet_json, path)
        except Exception as e:
            print('Enexpected Error: {}'.format(e))
            
    print('Iteration: {}\tFiles processed: {}'.format(i, len(unprocessed_tweets)))
    i+=1
    time.sleep(20)

New task was queued to ../shared_folder/geo_analyser_tasks/991648182802132992.task.txt
New task was queued to ../shared_folder/sentiment_tasks/991648182802132992.task.txt
New task was queued to ../shared_folder/user_tweets_tasks/838807021.task.txt
New task was queued to ../shared_folder/topic_analyser_tasks/991648182802132992.task.txt
New task was queued to ../shared_folder/geo_analyser_tasks/991647970763227136.task.txt
New task was queued to ../shared_folder/sentiment_tasks/991647970763227136.task.txt
New task was queued to ../shared_folder/user_tweets_tasks/167595513.task.txt
New task was queued to ../shared_folder/topic_analyser_tasks/991647970763227136.task.txt
New task was queued to ../shared_folder/geo_analyser_tasks/991648183326408704.task.txt
New task was queued to ../shared_folder/sentiment_tasks/991648183326408704.task.txt
New task was queued to ../shared_folder/user_tweets_tasks/3038998982.task.txt
New task was queued to ../shared_folder/topic_analyser_tasks/9916481833264087

New task was queued to ../shared_folder/geo_analyser_tasks/991647869651189760.task.txt
New task was queued to ../shared_folder/sentiment_tasks/991647869651189760.task.txt
New task was queued to ../shared_folder/user_tweets_tasks/699178838593396736.task.txt
New task was queued to ../shared_folder/topic_analyser_tasks/991647869651189760.task.txt
New task was queued to ../shared_folder/geo_analyser_tasks/991647890408751109.task.txt
New task was queued to ../shared_folder/sentiment_tasks/991647890408751109.task.txt
New task was queued to ../shared_folder/user_tweets_tasks/170238260.task.txt
New task was queued to ../shared_folder/topic_analyser_tasks/991647890408751109.task.txt
New task was queued to ../shared_folder/geo_analyser_tasks/991648123498807297.task.txt
New task was queued to ../shared_folder/sentiment_tasks/991648123498807297.task.txt
New task was queued to ../shared_folder/user_tweets_tasks/2524250539.task.txt
New task was queued to ../shared_folder/topic_analyser_tasks/9916481

New task was queued to ../shared_folder/topic_analyser_tasks/991648244752039936.task.txt
New task was queued to ../shared_folder/geo_analyser_tasks/991648209578541057.task.txt
New task was queued to ../shared_folder/sentiment_tasks/991648209578541057.task.txt
New task was queued to ../shared_folder/user_tweets_tasks/3192020780.task.txt
New task was queued to ../shared_folder/topic_analyser_tasks/991648209578541057.task.txt
Iteration: 2	Files processed: 11
New task was queued to ../shared_folder/geo_analyser_tasks/991648375970844672.task.txt
New task was queued to ../shared_folder/sentiment_tasks/991648375970844672.task.txt
New task was queued to ../shared_folder/user_tweets_tasks/816907107827974145.task.txt
New task was queued to ../shared_folder/topic_analyser_tasks/991648375970844672.task.txt
New task was queued to ../shared_folder/geo_analyser_tasks/991648297419915264.task.txt
New task was queued to ../shared_folder/sentiment_tasks/991648297419915264.task.txt
New task was queued to 

New task was queued to ../shared_folder/geo_analyser_tasks/991648612336586752.task.txt
New task was queued to ../shared_folder/sentiment_tasks/991648612336586752.task.txt
New task was queued to ../shared_folder/user_tweets_tasks/597029854.task.txt
New task was queued to ../shared_folder/topic_analyser_tasks/991648612336586752.task.txt
Iteration: 6	Files processed: 9
New task was queued to ../shared_folder/geo_analyser_tasks/991648715776507906.task.txt
New task was queued to ../shared_folder/sentiment_tasks/991648715776507906.task.txt
New task was queued to ../shared_folder/user_tweets_tasks/170238260.task.txt
New task was queued to ../shared_folder/topic_analyser_tasks/991648715776507906.task.txt
New task was queued to ../shared_folder/geo_analyser_tasks/991648700416978944.task.txt
New task was queued to ../shared_folder/sentiment_tasks/991648700416978944.task.txt
New task was queued to ../shared_folder/user_tweets_tasks/559935930.task.txt
New task was queued to ../shared_folder/topic_

New task was queued to ../shared_folder/geo_analyser_tasks/991648963571826688.task.txt
New task was queued to ../shared_folder/sentiment_tasks/991648963571826688.task.txt
New task was queued to ../shared_folder/user_tweets_tasks/407448653.task.txt
New task was queued to ../shared_folder/topic_analyser_tasks/991648963571826688.task.txt
New task was queued to ../shared_folder/geo_analyser_tasks/991648917224747009.task.txt
New task was queued to ../shared_folder/sentiment_tasks/991648917224747009.task.txt
New task was queued to ../shared_folder/user_tweets_tasks/626455551.task.txt
New task was queued to ../shared_folder/topic_analyser_tasks/991648917224747009.task.txt
New task was queued to ../shared_folder/geo_analyser_tasks/991648925592444930.task.txt
New task was queued to ../shared_folder/sentiment_tasks/991648925592444930.task.txt
New task was queued to ../shared_folder/user_tweets_tasks/362932624.task.txt
New task was queued to ../shared_folder/topic_analyser_tasks/99164892559244493

...

New task was queued to ../shared_folder/geo_analyser_tasks/991650047535169537.task.txt
New task was queued to ../shared_folder/sentiment_tasks/991650047535169537.task.txt
New task was queued to ../shared_folder/user_tweets_tasks/60526146.task.txt
New task was queued to ../shared_folder/topic_analyser_tasks/991650047535169537.task.txt
New task was queued to ../shared_folder/geo_analyser_tasks/991650046801199106.task.txt
New task was queued to ../shared_folder/sentiment_tasks/991650046801199106.task.txt
New task was queued to ../shared_folder/user_tweets_tasks/923782843154575360.task.txt
New task was queued to ../shared_folder/topic_analyser_tasks/991650046801199106.task.txt
New task was queued to ../shared_folder/geo_analyser_tasks/991650012433072133.task.txt
New task was queued to ../shared_folder/sentiment_tasks/991650012433072133.task.txt
New task was queued to ../shared_folder/user_tweets_tasks/15513006.task.txt
New task was queued to ../shared_folder/topic_analyser_tasks/9916500124

New task was queued to ../shared_folder/geo_analyser_tasks/991650162844954624.task.txt
New task was queued to ../shared_folder/sentiment_tasks/991650162844954624.task.txt
New task was queued to ../shared_folder/user_tweets_tasks/554326964.task.txt
New task was queued to ../shared_folder/topic_analyser_tasks/991650162844954624.task.txt
New task was queued to ../shared_folder/geo_analyser_tasks/991650214925647873.task.txt
New task was queued to ../shared_folder/sentiment_tasks/991650214925647873.task.txt
New task was queued to ../shared_folder/user_tweets_tasks/358132323.task.txt
New task was queued to ../shared_folder/topic_analyser_tasks/991650214925647873.task.txt
New task was queued to ../shared_folder/geo_analyser_tasks/991650190258941953.task.txt
New task was queued to ../shared_folder/sentiment_tasks/991650190258941953.task.txt
New task was queued to ../shared_folder/user_tweets_tasks/174602610.task.txt
New task was queued to ../shared_folder/topic_analyser_tasks/99165019025894195

New task was queued to ../shared_folder/geo_analyser_tasks/991650578643079168.task.txt
New task was queued to ../shared_folder/sentiment_tasks/991650578643079168.task.txt
New task was queued to ../shared_folder/user_tweets_tasks/1616540347.task.txt
New task was queued to ../shared_folder/topic_analyser_tasks/991650578643079168.task.txt
New task was queued to ../shared_folder/geo_analyser_tasks/991650564009115649.task.txt
New task was queued to ../shared_folder/sentiment_tasks/991650564009115649.task.txt
New task was queued to ../shared_folder/user_tweets_tasks/1582207812.task.txt
New task was queued to ../shared_folder/topic_analyser_tasks/991650564009115649.task.txt
New task was queued to ../shared_folder/geo_analyser_tasks/991650570854322177.task.txt
New task was queued to ../shared_folder/sentiment_tasks/991650570854322177.task.txt
New task was queued to ../shared_folder/user_tweets_tasks/3309547363.task.txt
New task was queued to ../shared_folder/topic_analyser_tasks/99165057085432

New task was queued to ../shared_folder/geo_analyser_tasks/991650887255736320.task.txt
New task was queued to ../shared_folder/sentiment_tasks/991650887255736320.task.txt
New task was queued to ../shared_folder/user_tweets_tasks/846279049.task.txt
New task was queued to ../shared_folder/topic_analyser_tasks/991650887255736320.task.txt
New task was queued to ../shared_folder/geo_analyser_tasks/991650844041871360.task.txt
New task was queued to ../shared_folder/sentiment_tasks/991650844041871360.task.txt
New task was queued to ../shared_folder/user_tweets_tasks/1294835346.task.txt
New task was queued to ../shared_folder/topic_analyser_tasks/991650844041871360.task.txt
New task was queued to ../shared_folder/geo_analyser_tasks/991650862584938497.task.txt
New task was queued to ../shared_folder/sentiment_tasks/991650862584938497.task.txt
New task was queued to ../shared_folder/user_tweets_tasks/71967898.task.txt
New task was queued to ../shared_folder/topic_analyser_tasks/99165086258493849

KeyboardInterrupt: 

# Copy tweets for reprocessing

In [ ]:
processed_tweets_path, tweets_queue_path

In [ ]:
#!for i in ../shared_folder/processed_tweets/*.json; do cp $i ../shared_folder/tweets/; done

## test

In [ ]:
with open('../shared_folder/processed_tweets/988986510979497985.json', 'r') as fp:
    data = json.load(fp)   
data['extended_tweet']['full_text']

In [ ]:
couch_db.update_document('988986510979497985', {'key1': 'dfsdf', 'key2': 'dsfjkds'})

In [ ]:
couch_db.save_tweet(data)

# Analyse tweets contents

In [ ]:
hashtags = ['bitcoin', 'blockchain', 'btc', 'cryptocurrency', 'market\svalue',
            'crypto','ethereum', 'fintech', 'coin', 'doge', 'ethereum', 'ripple', 'litecoin', 
            'cardano', 'monero', 'TRON', 'zcash', 'jaxx', 'copay', 'bitpay', 'exodus', 
            'mycelium', 'Bread\sWallet', 'trezor', 'ledger\snano', 'Silk\sRoad', 'darknet\smarket', 
            'dogecoin', 'ASIC\sMiner', 'Central\sLedger', 'Hashrate', 'ICO',         
           ]

In [ ]:
'|'.join(hashtags)

In [ ]:
bitcoin_pathes = []
for path in (unprocessed_tweets):
    with open(path, 'r') as fp:
        data = json.load(fp)
        if re.search('|'.join(hashtags), json.dumps(data)):
            bitcoin_pathes.append(path)
        #print(data['entities']['hashtags'])


In [ ]:
len(bitcoin_pathes)